In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
np.random.seed(132)
from functools import lru_cache

import sys

CODE_PATH = '../code'

sys.path.append(CODE_PATH)
import functions

from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, load_wine, load_digits
from scipy.optimize import minimize
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

%pylab inline

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Populating the interactive namespace from numpy and matplotlib


/home/basil/.local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


# Берем выборку

In [3]:
# dataset = load_breast_cancer()
dataset = load_digits()
df = pd.DataFrame(dataset['data'])
target = dataset['target']

# df = pd.read_csv('../data/model_36_12.csv')
# target = df['target'].astype(int).values
# del df['target']

from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.33, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(df.values, target, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape, X_val.shape)

(1005, 64) (360, 64) (432, 64)


# N случайных дихотомий (без отбора)

In [5]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 40 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        weight_type = None
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

Training dich classifiers:  10%|█         | 4/40 [00:00<00:00, 37.41it/s]

Code Matrix shape == (10,40)



Training dich classifiers:  15%|█▌        | 6/40 [00:00<00:00, 54.24it/s]

Code Matrix shape == (10,40)



Training dich classifiers:  12%|█▎        | 5/40 [00:00<00:00, 48.00it/s]

...


Training dich classifiers:  15%|█▌        | 6/40 [00:00<00:00, 51.75it/s]

Code Matrix shape == (10,40)



Training dich classifiers:  15%|█▌        | 6/40 [00:00<00:00, 54.85it/s]

Code Matrix shape == (10,40)



100%|██████████| 5/5 [00:12<00:00,  2.46s/it]


In [6]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.0
mean,0.905556,0.909259,0.909259,0.912500,40.0
std,0.006629,0.007209,0.005764,0.003018,0.0
min,0.898148,0.900463,0.900463,0.909722,40.0
25%,0.900463,0.902778,0.907407,0.909722,40.0
50%,0.905093,0.912037,0.909722,0.912037,40.0
75%,0.909722,0.914352,0.914352,0.914352,40.0
max,0.914352,0.916667,0.914352,0.916667,40.0


# Отбор

In [7]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 80 # кол-во дихотомий
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
#     print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    for score_type in wtypes:
        preds, num_real_dich = functions.predict_all(X_val, dich_classifiers, 
                                      code_matrix, score_type, weight_type=-1)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)

100%|██████████| 5/5 [00:25<00:00,  5.02s/it]0/80 [00:01<00:00, 48.97it/s]


In [8]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.000000
mean,0.918519,0.920833,0.920833,0.916667,39.800000
std,0.005018,0.005279,0.005018,0.003274,3.271085
min,0.912037,0.914352,0.914352,0.912037,35.000000
25%,0.916667,0.916667,0.918981,0.916667,38.000000
50%,0.916667,0.921296,0.918981,0.916667,41.000000
75%,0.923611,0.925926,0.925926,0.916667,42.000000
max,0.923611,0.925926,0.925926,0.921296,43.000000


# Дихотомии где остались лучшие по критерию

In [10]:
attempts_data = []
N_attempts = 5
l = np.unique(target).size
N = 80 # кол-во дихотомий
num_real_dich = 40 # кол-во дихотомий которые оставляем
wtypes = [None, 'accuracy', 'f1', 'confusion_list']
for i in tqdm(range(N_attempts)):
    accs = []
    code_matrix = functions.make_random_dichs(l, N)
    print('Code Matrix shape == ({},{})'.format(l, code_matrix.shape[1]))
    dich_classifiers = functions.train_dichs(code_matrix, X_train, y_train, 
                                             X_test, y_test, LogisticRegression)
    
    metric_map = np.array([d['accuracy'] for d in dich_classifiers])
    top_dich = np.sort(np.argsort(metric_map)[-num_real_dich:])
    dich_classifiers = [d for i,d in enumerate(dich_classifiers) if i in top_dich]
    code_matrix = code_matrix.T[top_dich].T
    
    for score_type in wtypes:
        weight_type = None
        preds, _ = functions.predict_all(X_val, dich_classifiers, code_matrix, score_type, weight_type)
        acc = accuracy_score(preds, y_val)
        accs.append(acc)
    accs.append(num_real_dich)
    attempts_data.append(accs)
#             print(score_type, weight_type, accuracy_score(preds, y_val))

Training dich classifiers:   5%|▌         | 4/80 [00:00<00:02, 32.85it/s]

Code Matrix shape == (10,80)



Training dich classifiers:   9%|▉         | 7/80 [00:00<00:01, 59.59it/s]

Code Matrix shape == (10,80)



Training dich classifiers:   8%|▊         | 6/80 [00:00<00:01, 50.78it/s]

...


Training dich classifiers:   8%|▊         | 6/80 [00:00<00:01, 53.06it/s]

Code Matrix shape == (10,80)



Training dich classifiers:   8%|▊         | 6/80 [00:00<00:01, 54.06it/s]

Code Matrix shape == (10,80)



100%|██████████| 5/5 [00:16<00:00,  3.39s/it]0/80 [00:01<00:00, 47.69it/s]


In [11]:
wtypes += ['num_real_dich']
df_attempts = pd.DataFrame(attempts_data)
df_attempts.columns = ['s{}'.format(i1) for i1 in wtypes]
df_attempts.describe()

,sNone,saccuracy,sf1,sconfusion_list,snum_real_dich
count,5.000000,5.000000,5.000000,5.000000,5.0
mean,0.921296,0.923148,0.921759,0.921759,40.0
std,0.010352,0.012183,0.012073,0.009600,0.0
min,0.909722,0.912037,0.914352,0.912037,40.0
25%,0.914352,0.916667,0.914352,0.916667,40.0
50%,0.918981,0.916667,0.914352,0.916667,40.0
75%,0.928241,0.928241,0.923611,0.928241,40.0
max,0.935185,0.942130,0.942130,0.935185,40.0


# Локальный метод оптимизации

In [ ]:
def score_function(cur_dich, X_train, y_train, X_test, y_test):
    # cluster
    target = np.array([cur_dich[i] for i in y_train])
    return -functions.cluster_score(X_train, target, score_type='trace_w')

In [ ]:
def score_function(cur_dich, X_train, y_train, X_test, y_test):
    # accuracy
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [ ]:
def score_function(cur_dich, X_train, y_train, X_test, y_test):
    # f1
    X = X_train
    y = np.array([cur_dich[i] for i in y_train])
    clf = LogisticRegression()
    clf.fit(X, y)
    y_pred = clf.predict(X_test)
    y_true = np.array([cur_dich[i] for i in y_test])
    accuracy = f1_score(y_true, y_pred)
    return accuracy

In [ ]:
cur_dich = code_matrix.T[1].copy()
cur_score = score_function(cur_dich, X_train, y_train, X_test, y_test)
next_score = cur_score
cur_score

In [ ]:
while True:
    next_dich = cur_score.copy()
    next_scores = np.zeros(len(cur_dich)) - 1
    for i in range(len(cur_dich)):
        next_dich = cur_dich.copy()
        next_dich[i] = 1 - next_dich[i]
        if not functions.does_dich_exist(next_dich, code_matrix): #дихотомия нормальная
            next_scores[i] = score_function(next_dich, X_train, y_train, X_test, y_test)
    next_scores = np.array(next_scores)
    next_score = next_scores.max()
    if next_score <= cur_score: #идем только на повышение, но можно скор сделать поменьше
        break
    cur_score = next_score
    best_index = np.random.choice(np.flatnonzero(next_scores == next_score)) # it is random of the best
    print(next_score, best_index)
    print(cur_dich)
    cur_dich[best_index] = 1 - cur_dich[best_index]

In [ ]:
cur_dich